In [1]:
import re
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [2]:
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
path_raw = "../data/external/subdere"

## DataFrame Provincias de Chile  - Fuente: Subdere

In [4]:
df_provincias = pd.read_excel(f"{path_raw}/provinciasChile.xls")

In [5]:
df_provincias

,Código Región,Nombre Región,Código Provincia,Nombre Provincia,Código Comuna 2010,Nombre Comuna
0,1,Tarapacá,11,Iquique,1101,Iquique
1,1,Tarapacá,11,Iquique,1107,Alto Hospicio
2,1,Tarapacá,14,Tamarugal,1401,Pozo Almonte
3,1,Tarapacá,14,Tamarugal,1402,Camiña
4,1,Tarapacá,14,Tamarugal,1403,Colchane
...,...,...,...,...,...,...
341,14,Los Ríos,142,Ranco,14204,Río Bueno
342,15,Arica y Parinacota,151,Arica,15101,Arica
343,15,Arica y Parinacota,151,Arica,15102,Camarones
344,15,Arica y Parinacota,152,Parinacota,15201,Putre


In [6]:
df_provincias.dtypes

Código Región          int64
Nombre Región         object
Código Provincia       int64
Nombre Provincia      object
Código Comuna 2010     int64
Nombre Comuna         object
dtype: object

In [7]:
# Transformo a mayusculas las columnas de mi interes
df_provincias['Nombre Región'] = df_provincias['Nombre Región'].str.upper()
df_provincias['Nombre Provincia'] = df_provincias['Nombre Provincia'].str.upper()
df_provincias['Nombre Comuna'] = df_provincias['Nombre Comuna'].str.upper()

cols = df_provincias.select_dtypes(include = ["object"]).columns
df_provincias[cols] = df_provincias[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 3/3 [00:00<00:00, 209.98it/s]


In [8]:
# Cambio nombre provincia para coincidir con otro df

df_provincias.loc[df_provincias['Nombre Provincia'] == 'ANTARTICA CHILENA', 'Nombre Provincia'] = 'LA ANTARTICA CHILENA'
df_provincias.loc[df_provincias['Nombre Comuna'] == 'COIHAYQUE', 'Nombre Comuna'] = 'COIHAIQUE'
df_provincias.loc[df_provincias['Nombre Comuna'] == 'PAIGUANO', 'Nombre Comuna'] = 'PAIHUANO'
df_provincias.loc[df_provincias['Nombre Comuna'] == 'TILTIL', 'Nombre Comuna'] = 'TIL TIL'
df_provincias.loc[df_provincias['Nombre Comuna'] == 'EL OLIVAR', 'Nombre Comuna'] = 'OLIVAR'

In [9]:
# Reset el index para realizar feather

df_provincias.reset_index(inplace = True)

path_interim = "../data/interim/subdere"
os.makedirs(path_interim, exist_ok = True) 

# Guardamos dataset como archivo feather
df_provincias.to_feather(f'{path_interim}/Provincias_feather')
